# PJM Hourly Energy Consumption Case

PJM Interconnection LLC (PJM) is a regional transmission organization (RTO) in the United States. It is part of the Eastern Interconnection grid operating an electric transmission system serving all or parts of Delaware, Illinois, Indiana, Kentucky, Maryland, Michigan, New Jersey, North Carolina, Ohio, Pennsylvania, Tennessee, Virginia, West Virginia, and the District of Columbia.

The hourly power consumption data comes from PJM's website and are in megawatts (MW).

### LSTM Training Step - By Sabrina Otoni da Silva - 2024/04

In [2]:
from pathlib import Path 

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit

import tensorflow.keras.layers as L
from keras_tuner import Hyperband
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping

import pickle

Using TensorFlow backend


In [3]:
datapath = Path('../data/d02_intermediate')
modelpath = Path('../model')

In [4]:
df = pd.read_csv(f'{datapath}/pjme_train.csv')
df.set_index('datetime', inplace=True)
df.index = pd.to_datetime(df.index)
df.sort_index(inplace=True)
df.dropna(axis=0, how='any', inplace=True)

In [5]:
def temporalize(X, y, lookback):
    '''
    To convert input data into 3-D
    array as required for LSTM network.
    '''
    output_X = []
    output_y = []
    for i in range(len(X)-lookback-1):
        t = []
        for j in range(1,lookback+1):
            t.append(X[[(i+j+1)], :])
        output_X.append(t)
        output_y.append(y[i+lookback+1])
        
    return output_X, output_y

In [6]:
def flatten(X):
    '''
    Flatten a 3D array.
    Input
    X - A 3D array for lstm, where the array is sample x timesteps x features.

    Output
    flattened_X - A 2D array, sample x features.
    '''
    flattened_X = np.empty((X.shape[0], X.shape[2]))  # Sample x features array.
    for i in range(X.shape[0]):
        flattened_X[i] = X[i, (X.shape[1]-1), :]
    return(flattened_X)

def scale(X, scaler):
    '''
    Scale 3D array.
    Inputs
    X - A 3D array for lstm, where the array is sample x timesteps x features.
    scaler - A scaler object, e.g., sklearn.preprocessing.StandardScaler, sklearn.preprocessing.normalize
    
    Output
    X - Scaled 3D array.
    '''
    for i in range(X.shape[0]):
        X[i, :, :] = scaler.transform(X[i, :, :])
        
    return X

In [7]:
n_features = df.shape[1] - 1
timesteps = 24

X_train, y_train = temporalize(X = np.array(df[['hour', 'dayofweek', 'quarter', 'month', 'year', 'dayofyear', 'day', 'weekofyear', 'lag1', 'lag2', 'lag3']]), 
                   y = np.array(df[['pjme_mw']]), 
                   lookback = timesteps)

X_train = np.array(X_train)
X_train = X_train.reshape(X_train.shape[0], timesteps, n_features)
y_train = np.array(y_train)

In [8]:
scaler_x = StandardScaler().fit(flatten(X_train))
X_train = scale(X_train, scaler_x)

In [9]:
tscv = TimeSeriesSplit(n_splits=5, test_size=24*365*1, gap=24)

In [10]:
# def build_model(hp):
#     model = Sequential()
#     # Encoder
#     model.add(L.LSTM(hp.Int('encoder_lstm_1_units', min_value=64, max_value=256, step=32),
#                      activation='relu',
#                      input_shape=(timesteps, n_features),
#                      return_sequences=True))
#     model.add(L.LSTM(hp.Int('encoder_lstm_2_units', min_value=32, max_value=128, step=32),
#                      activation='relu',
#                      return_sequences=False))
#     model.add(L.RepeatVector(timesteps))
#     # Decoder
#     model.add(L.LSTM(hp.Int('decoder_lstm_1_units', min_value=32, max_value=128, step=32),
#                      activation='relu',
#                      return_sequences=True))
#     model.add(L.LSTM(hp.Int('decoder_lstm_2_units', min_value=64, max_value=256, step=32),
#                      activation='relu',
#                      return_sequences=True))
#     model.add(L.TimeDistributed(L.Dense(1, activation='linear')))
    
#     # Compiling the model
#     model.compile(optimizer=optimizers.Adam(
#                       hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
#                   loss='mse')
    
#     return model

In [11]:
# best_score = float('inf')
# best_model = None

# for fold, (train_index, val_index) in enumerate(tscv.split(X_train)):
#     print(f"Training on fold {fold+1}...")
#     X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
#     y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]
    
#     tuner = Hyperband(
#         build_model,
#         objective='val_loss',
#         max_epochs=40,
#         hyperband_iterations=2,
#         factor=3,
#         directory=f'{modelpath}/keras_tuner',
#         project_name='lstm_autoencoder'
#     )
    
#     early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

#     tuner.search(
#         x=X_train_fold,
#         y=y_train_fold,
#         epochs=50,
#         validation_data=(X_val_fold, y_val_fold),
#         callbacks=[early_stopping],
#         verbose=5
#     )

#     best_model_fold = tuner.get_best_models(num_models=1)[0]
#     best_hyperparameters_fold = tuner.get_best_hyperparameters(num_trials=1)[0]
    
#     val_score_fold = best_model_fold.evaluate(X_val_fold, y_val_fold)

#     if val_score_fold < best_score:
#         best_score = val_score_fold
#         best_model = best_model_fold

#     best_model.save(f'{modelpath}/best_model.h5')
    
#     with open(f'{modelpath}/best_hyperparameters.pkl', 'wb') as f:
#         pickle.dump(best_hyperparameters_fold, f)
    
#     print(f"Validation score for fold {fold+1}: {val_score_fold}")

In [12]:
def lstm_autoencoder():
    model = Sequential()
    # Encoder
    model.add(L.LSTM(128, activation='relu', input_shape=(timesteps, n_features), return_sequences=True))
    model.add(L.LSTM(64, activation='relu', return_sequences=False))
    model.add(L.RepeatVector(timesteps))
    # Decoder
    model.add(L.LSTM(64, activation='relu', return_sequences=True))
    model.add(L.LSTM(128, activation='relu', return_sequences=True))
    model.add(L.TimeDistributed(L.Dense(1, activation='linear')))
    
    model.compile(optimizer=Adam(0.001), loss='mse')
    
    return model

In [13]:
lstm_autoencoder = lstm_autoencoder()
lstm_autoencoder.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 24, 128)           71680     
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 repeat_vector (RepeatVecto  (None, 24, 64)            0         
 r)                                                              
                                                                 
 lstm_2 (LSTM)               (None, 24, 64)            33024     
                                                                 
 lstm_3 (LSTM)               (None, 24, 128)           98816     
                                                                 
 time_distributed (TimeDist  (None, 24, 1)             129       
 ributed)                                               

In [14]:
# for fold, (train_index, val_index) in enumerate(tscv.split(X_train)):
#     print(f"Training on fold {fold+1}...")
#     X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
#     y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

history = lstm_autoencoder.fit(
        X_train, y_train,
        epochs=10,
        batch_size=113919,
        #validation_data=(X_val_fold, y_val_fold),
        verbose=1
    )

    # val_loss = lstm_autoencoder.evaluate(X_val_fold, y_val_fold)
    # print("Validation Loss:", val_loss)

Epoch 1/10


In [ ]:
plt.plot(history.history['loss'], label='Train Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()